In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingClassifier
from sklearn import  metrics
from sklearn.model_selection import GridSearchCV

# 2. Ансамбли моделей. Бутстреппинг. Бэггинг

In [2]:
# 2.2
# Вычислите дисперсию для этого ряда. 
x = [4.4, 3.8, 4.8, 4.0, 3.4, 4.2, 5.2]
# Результат округлите до трёх знаков после точки-разделителя.
m = sum(x)/len(x)
y = []
for z in x:
    y.append((z-m)**2)
round((sum(y)/len(y)),3)

0.317

### BIAS И VARIANCE

In [3]:
# 2.7
# Объёмная и содержательная практика у нас ещё впереди, но в качестве разминки давайте поработаем с уже известным вам датасетом о вине, 
# который можно скачать здесь.

# Ранее вы обучали на данных только один алгоритм, а теперь мы попробуем сравнить несколько.

# Подготовьте данные к классификации. Условно разделите вино на хорошее и плохое. 
# Хорошим вином будем называть то, параметр quality которого — 6 и более.
# Сравните несколько методов классификации: логистическую регрессию, дерево решений и бэггинг. 
# Это позволит вам увидеть, как меняется качество в зависимости от выбора того или иного алгоритма.
# Разделите выборку на обучающую и тестовую в соотношении 70/30, в качестве значения параметра random_state возьмите число 42.
# Для начала обучите два классификатора: логистическую регрессию (с параметрами по умолчанию) и дерево решений 
# (random_state = 42, максимальная глубина — 10).
# 1. Введите значение F1-score для классификатора, который показал наилучшее значение. 
# Ответ округлите до трёх знаков после точки-разделителя.

df = pd.read_csv('data/wineQualityReds.csv', index_col="Unnamed: 0")
df['class']= df['quality'].apply(lambda x: 1 if x>5 else 0)
X = df.drop(['quality','class'], axis=1)
y = df['class']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=42)

log = LogisticRegression()
log.fit(X_train, y_train)
y_pred_log = log.predict(X_test)
print("log f1", round(metrics.f1_score(y_test, y_pred_log),3))

dt = DecisionTreeClassifier(
    random_state=42,
    max_depth=10
)
dt.fit(X_train, y_train)
y_pred_dt= dt.predict(X_test)
print("dt f1", round(metrics.f1_score(y_test, y_pred_dt),3))

log f1 0.753
dt f1 0.793


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [4]:
# 2.8
# Обучите модель с использованием бэггинга (класс BaggingClassifier с random_state=42).

# Возьмите из предыдущего задания алгоритм, показавший наилучшее качество, и укажите для него новое количество моделей — 1500. 
# Вычислите новое значение F1-score.

bag = BaggingClassifier(
    base_estimator=dt,
    n_estimators=1500,
    random_state=42
)
bag.fit(X_train, y_train)
y_pred_bag= bag.predict(X_test)
print("bag f1", round(metrics.f1_score(y_test, y_pred_bag),3))

bag f1 0.818


# 3. Случайный лес

In [5]:
# 3.4
# 1. Разбейте набор данных на обучающую и тестовую выборку в соотношении 70/30, при разбиении задайте параметр random_state = 13.

# Какое получилось среднее значение медианных цен на обучающей выборке? Ответ округлите до двух знаков после точки-разделителя.

df = pd.read_csv('data/boston.csv')

# заменяем везде запятые (,) на точки (.)для конвертации во float
col = df.columns
for c in col:
    if c not in ['river', 'highways_index', 'tax']:
        df[c] = df[c].str.replace(',','.').astype(float)

X = df.drop(['target'], axis=1)
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=13)
print("1: ", round(y_train.mean(),2))

# 2. Обучите линейную регрессию с параметрами по умолчанию.

# В качестве ответа введите ошибку MAE на тестовой выборке. Ответ округлите до двух знаков после точки-разделителя.

lin = LinearRegression()
lin.fit(X_train, y_train)
y_pred_lin= lin.predict(X_test)
print("lin MAE", round(metrics.mean_absolute_error(y_test, y_pred_lin),2))

# 3. Обучите решающее дерево с параметрами по умолчанию и аргументом random_state = 13.

# Можно ли, опираясь на результаты, сделать вывод, что алгоритм переобучился?

dt = DecisionTreeRegressor(
    random_state=42
)
dt.fit(X_train, y_train)
y_pred_dt_train= dt.predict(X_train)
y_pred_dt= dt.predict(X_test)
print("dt train", round(metrics.mean_absolute_error(y_train, y_pred_dt_train),2))
print("dt test", round(metrics.mean_absolute_error(y_test, y_pred_dt),2))

# 4. Обучите четыре случайных леса с числом деревьев 3, 10, 100, 500 и параметром random_state = 13.

#В качестве ответа введите наименьшую полученную ошибку MAE на тестовой выборке. 
# Ответ округлите до двух знаков после точки-разделителя.

for n in [3, 10, 100, 500]:
    model = RandomForestRegressor(n_estimators=n, random_state = 13)
    model.fit(X_train, y_train)
    preds_train = model.predict(X_train)
    preds_test = model.predict(X_test)
    print(metrics.mean_absolute_error(y_test, preds_test))

1:  22.77
lin MAE 3.72
dt train 0.0
dt test 2.98
2.9328947368421057
2.466315789473684
2.2559736842105265
2.2365657894736892


# 4. Случайный лес. Практика

In [2]:
data = pd.read_csv("data/weatherAUS.csv")

# 4.1 
# Сколько суммарно пропусков в данных?
data.isnull().sum().sum()

343248

In [3]:
# 4.2
# В некоторых признаках пропусков более 40 % — удалите такие признаки. Сколько их было?
dd = data.isnull().sum()/len(data)*100
print(dd.sort_values(ascending=False))
data = data.drop(['Evaporation','Sunshine','Cloud3pm'], axis = 1)

Sunshine         48.009762
Evaporation      43.166506
Cloud3pm         40.807095
Cloud9am         38.421559
Pressure9am      10.356799
Pressure3pm      10.331363
WindDir9am        7.263853
WindGustDir       7.098859
WindGustSpeed     7.055548
Humidity3pm       3.098446
WindDir3pm        2.906641
Temp3pm           2.481094
RainTomorrow      2.245978
Rainfall          2.241853
RainToday         2.241853
WindSpeed3pm      2.105046
Humidity9am       1.824557
Temp9am           1.214767
WindSpeed9am      1.214767
MinTemp           1.020899
MaxTemp           0.866905
Location          0.000000
Date              0.000000
dtype: float64


In [4]:
# 4.3
# Теперь обработаем признаки RainToday и RainTomorrow таким образом, чтобы вместо yes было значение 1, а вместо no — значение 0. 
# Обратите внимание на то, что в признаке RainTomorrow присутствуют пропуски, и их трогать не нужно, они должны остаться пропусками. 
# Поэтому обрабатывайте столбцы таким образом, чтобы не видоизменить пропущенные значения.

# Вычислите среднее арифметическое для преобразованного признака RainToday и запишите его в ответ, 
# предварительно округлив до двух знаков после точки-разделителя.

data['RainToday'] = data['RainToday'].apply(lambda x: 1 if (x and x=='Yes') else 0)
data['RainTomorrow'] = data['RainTomorrow'].apply(lambda x: 1 if (x and x=='Yes') else 0)
round(data['RainToday'].mean(),2)

# df.RainToday = df.RainToday.map({'No': 0, 'Yes': 1})
# df.RainTomorrow = df.RainTomorrow.map({'No': 0, 'Yes': 1})


0.22

In [5]:
# 4.4
# Обработайте признак Date таким образом, чтобы выделить в отдельный признак Month (номер месяца). 
# Изначальный признак Date удалите. Определите, в какой месяц в среднем за день выпадает больше всего дождей. 
# В качестве ответа введите порядковый номер месяца.
data['Date'] = pd.to_datetime(data['Date'])
data['Month'] = data['Date'].dt.month
data = data.drop(['Date'], axis=1)
data.groupby(['Month'])['RainToday'].mean().sort_values(ascending=False)

Month
7     0.265198
6     0.257569
8     0.247589
9     0.223368
5     0.217255
4     0.212208
3     0.211885
12    0.207699
11    0.206959
2     0.201612
10    0.192966
1     0.184875
Name: RainToday, dtype: float64

In [6]:
# 4.5
# Обработайте оставшиеся категориальные признаки. 
# С помощью метода get_dummies с настройками по умолчанию создайте dummy-переменные для всех категориальных признаков (их пять), 
# которые есть в данных на этот момент.
# Кодировку признаков важно выполнить именно в следующем порядке: 
# categoricals = ['Month', 'Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm']. 
# Это необходимо для того, чтобы ваши дальнейшие ответы сходились с нашим решением, 
# так как алгоритм случайного леса, который мы будем использовать в дальнейшем, чувствителен к порядку столбцов.

# Сколько теперь признаков в данных, если считать целевую переменную?
categoricals = ['Month', 'Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm']
data_hot = pd.get_dummies(data, columns= categoricals)
data_hot.shape

(145460, 124)

In [7]:
# 4.6
# Осталось совсем немного. Удалите все строки, где есть пропуски. 
# Далее разбейте данные на обучающую и тестовую выборки в соотношении 70/30, 
# в качестве значения параметра random_state возьмите число 31.

# Каково среднее значение целевой переменной на тестовой выборке? 
# Ответ округлите до двух знаков после точки-разделителя.

data_hot = data_hot.dropna(axis=0)
X = data_hot.drop(['RainTomorrow'], axis=1)
y = data_hot['RainTomorrow']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=31)
round(y_test.mean(),2)

0.23

In [8]:
# 4.7
# # Теперь давайте вспомним про бутстреп. Он не понадобится нам для решения этой задачи, но будет полезно реализовать его «вручную».

# Сделайте оценку стандартного отклонения для среднего значения минимальной температуры для обучающей выборки 
# (то есть для среднего значения по признаку MinTemp). 
# Для этого сгенерируйте 1000 случайных выборок из наших данных — каждая из них должна быть такого же объёма, 
# как и обучающая выборка. Для генерации выборки используйте np.random.randint(): 
# сгенерируйте необходимое количество индексов и по ним извлеките соответствующие элементы выборки. 
# Случайность фиксируйте с помощью np.random.seed(31).

# Для каждой выборки вычислите среднее значение, а после найдите стандартное отклонение для этих значений. 
# Ответ округлите до двух знаков после точки-разделителя.
import numpy as np

np.random.seed(31)
n = X_train.shape[0] # размер выборки
h = X.shape[0] # максимальный возможный индекс (для создания выборки)
mean_temp = []
for _ in range(1000):
    ind = np.random.randint(low=0,high=h, size=n)
    select = data_hot.iloc[ind]
    mean_temp.append(select['MinTemp'].mean())

round(np.std(mean_temp),2)


0.03

In [9]:
# 4.8
# Теперь можно перейти к обучению прогностических моделей. 
# Начнём с того, что построим простейшую логистическую регрессию (без настройки гиперпараметров). 
# Это будет та модель, с качеством которой мы будем сравнивать результаты, полученные далее, 
# чтобы оценить превосходство случайного леса над простыми методами.

# В качестве ответа введите значение метрики roc_auc на тестовой выборке. 
# Ответ округлите до двух знаков после точки-разделителя.

log = LogisticRegression()
log.fit(X_train, y_train)
y_pred_log = log.predict(X_test)
print(metrics.roc_auc_score(y_test, y_pred_log))

0.7239606676472516


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [16]:
# 4.9
# Теперь попробуйте обучить на наших данных другой алгоритм — дерево решений. 
# С помощью GridSearchCV сделайте перебор гиперпараметров по следующей сетке:
params = {'max_leaf_nodes': list(range(2, 10)), 'min_samples_split': [2, 3, 4], 'max_depth': [5,7,9,11]}
# Для параметра кросс-валидации cv задайте значение 3. Для решающего дерева определите параметр random_state=42. 
# Остальные параметры оставьте по умолчанию.
dt = DecisionTreeClassifier(
    random_state=42
)

grid = GridSearchCV(
    dt,
    param_grid=params,
    cv=3
)
grid.fit(X_train,y_train)
best_tree = grid.best_estimator_
# 1. Вычислите значение roc_auc для решающего дерева с гиперпараметрами, определёнными в качестве оптимальных. 
# Ответ округлите до двух знаков после точки-разделителя.
y_pred_best_tree = best_tree.predict(X_test)
print("1 roc_auc:", metrics.roc_auc_score(y_test, y_pred_best_tree))

# 2. Какие значения гиперпараметров были выбраны как оптимальные?
print(grid.best_params_)

1 roc_auc: 0.7077676272152034
{'max_depth': 5, 'max_leaf_nodes': 8, 'min_samples_split': 2}


In [18]:
# 4.10
# К сожалению, деревья решений не помогли нам в улучшении качества модели, так что попробуем ещё уменьшить ошибку с помощью ансамблей.

# Теперь постройте случайный лес, включающий 100 деревьев. Задайте параметр random_state=31. Остальные параметры оставьте по умолчанию.

# Какой теперь будет метрика roc_auc на тестовой выборке? Ответ округлите до двух знаков после точки-разделителя.

rf = RandomForestRegressor(
    n_estimators=100,
    random_state=31
)

rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print(metrics.roc_auc_score(y_test, y_pred_rf))

0.8847930332710492


In [19]:
# 4.11
# Основные параметры, которые отвечают за качество обучения в случайном лесе, следующие:'max_features', 'min_samples_leaf', 'max_depth'.

# Возьмите случайный лес из 100 деревьев и найдите оптимальную комбинацию этих трёх параметров. Сетка для перебора следующая:

# {'max_features': [ 4, 5, 6, 7], 'min_samples_leaf': [3, 5, 7, 9, 11], 'max_depth': [5, 10, 15]}
# Перебор осуществите с помощью GridSearchCV. Для параметра кросс-валидации cv задайте значение 3. 
# Случайности фиксируйте параметром random_state = 31. Остальные значения оставьте по умолчанию.

# Какое значение roc_auc получилось для оптимальных гиперпараметров?

rf_params = {'max_features': [ 4, 5, 6, 7], 'min_samples_leaf': [3, 5, 7, 9, 11], 'max_depth': [5, 10, 15]}
rf_grid = GridSearchCV(
    rf,
    param_grid=rf_params,
    cv=3
)

rf_grid.fit(X_train,y_train)
best_forest = rf_grid.best_estimator_

y_pred_best_forest = best_forest.predict(X_test)
print("roc_auc:", metrics.roc_auc_score(y_test, y_pred_best_forest))


roc_auc: 0.8730351033620481


In [27]:
# 4.12
# Как мы говорили в предыдущем юните, благодаря случайному лесу можно узнать, 
# какие признаки оказывают большее влияние на целевую переменную по сравнению с другими.

# Оцените значимость признаков. Отметьте три признака, которые дают наибольший вклад в целевую переменную:
feature_names = [x for x in data_hot if x != 'RainTomorrow']
pd.DataFrame({'feat': feature_names,
              'coef': best_forest.feature_importances_}).sort_values(by='coef', ascending=False)

,feat,coef
7,Humidity3pm,0.251780
2,Rainfall,0.081177
10,Cloud9am,0.065202
9,Pressure3pm,0.065152
8,Pressure9am,0.061867
...,...,...
62,Location_SalmonGums,0.000000
26,Location_Adelaide,0.000000
40,Location_GoldCoast,0.000000
49,Location_MountGinini,0.000000


# 5. Бустинг

# 6. Бустинг. Практика

In [31]:
data = pd.read_csv('data/AirPass.csv')
data.drop(['Unnamed: 0'],axis=1, inplace=True)
data

,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,3,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,3,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
3,24026,Female,Loyal Customer,25,Business travel,Business,562,2,5,5,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,119299,Male,Loyal Customer,61,Business travel,Business,214,3,3,3,...,3,3,4,4,3,3,3,0,0.0,satisfied
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103899,94171,Female,disloyal Customer,23,Business travel,Eco,192,2,1,2,...,2,3,1,4,2,3,2,3,0.0,neutral or dissatisfied
103900,73097,Male,Loyal Customer,49,Business travel,Business,2347,4,4,4,...,5,5,5,5,5,5,4,0,0.0,satisfied
103901,68825,Male,disloyal Customer,30,Business travel,Business,1995,1,1,1,...,4,3,2,4,5,5,4,7,14.0,neutral or dissatisfied
103902,54173,Female,disloyal Customer,22,Business travel,Eco,1000,1,1,1,...,1,4,5,1,5,4,1,0,0.0,neutral or dissatisfied


In [32]:
# 6.1
# Для начала сделаем небольшую предобработку данных. Сколько всего в данных пропущенных значений?
data.isnull().sum().sum()

310

In [37]:
# 6.2
# Теперь давайте избавимся от найденных пропусков. Заполните их все медианными значениями. 
# После этого вычислите среднее арифметическое для признака, отражающего задержку при прибытии в минутах. 
# Ответ округлите до двух знаков после точки-разделителя.

display(data.isnull().sum())
data['Arrival Delay in Minutes'] = data['Arrival Delay in Minutes'].fillna(data['Arrival Delay in Minutes'].median())
data['Arrival Delay in Minutes'].mean()

id                                     0
Gender                                 0
Customer Type                          0
Age                                    0
Type of Travel                         0
Class                                  0
Flight Distance                        0
Inflight wifi service                  0
Departure/Arrival time convenient      0
Ease of Online booking                 0
Gate location                          0
Food and drink                         0
Online boarding                        0
Seat comfort                           0
Inflight entertainment                 0
On-board service                       0
Leg room service                       0
Baggage handling                       0
Checkin service                        0
Inflight service                       0
Cleanliness                            0
Departure Delay in Minutes             0
Arrival Delay in Minutes             310
satisfaction                           0
dtype: int64

15.133392362180475

In [66]:
# 6.3
# Проведём небольшой разведывательный анализ. Посмотрим, в каких категориях пассажиров превалировали удовлетворённые полетом клиенты.

# Совет: для ответов на вопросы попробуйте использовать как вычисления, так и визуализации.

# 1. Сравните удовлетворённость полётом мужчин и женщин. Выберите верное утверждение:
display(data.groupby(['Gender','satisfaction'])['id'].count())

# 2. Теперь сравним категории пассажиров в зависимости от их цели поездки. Выберите все верные утверждения:
display(data.groupby(['Type of Travel','satisfaction'])['id'].count())

# 3. В каком туристическом классе наибольший процент довольных клиентов?
display(data.groupby(['Class'])['satisfaction'].value_counts(normalize=True))

Gender  satisfaction           
Female  neutral or dissatisfied    30193
        satisfied                  22534
Male    neutral or dissatisfied    28686
        satisfied                  22491
Name: id, dtype: int64

Type of Travel   satisfaction           
Business travel  neutral or dissatisfied    29909
                 satisfied                  41746
Personal Travel  neutral or dissatisfied    28970
                 satisfied                   3279
Name: id, dtype: int64

Class     satisfaction           
Business  satisfied                  0.694251
          neutral or dissatisfied    0.305749
Eco       neutral or dissatisfied    0.813862
          satisfied                  0.186138
Eco Plus  neutral or dissatisfied    0.753936
          satisfied                  0.246064
Name: satisfaction, dtype: float64

In [67]:
# Перекодируем часть бинарных признаков, чтобы использовать их при обучении:
df = data.copy()
df['satisfaction'] = df['satisfaction'].map({'neutral or dissatisfied':0 , 'satisfied':1})
df['Customer Type'] = df['Customer Type'].map({'Loyal Customer':1, 'disloyal Customer':0})
df['Type of Travel'] = df['Type of Travel'].map({'Personal Travel':0, 'Business travel':1})
df['Gender'] = df['Gender'].map({'Male': 0, 'Female': 1})

In [73]:
# 6.4
# Для остальных категориальных признаков создайте dummy-переменные. 
# Сделайте это с помощью функции get_dummies() из библиотеки Pandas, параметры не меняйте. 
# Сколько теперь признаков в данных (включая целевую переменную)?


# Сброс ограничений на число столбцов
pd.set_option('display.max_columns', None)
df=pd.get_dummies(df)
df.shape

(103904, 26)

In [76]:
# 6.5
# Мы практически добрались до обучения модели. 
# Разбейте данные на обучающую и тестовую выборки в соотношении 80/20, параметр random_state = 26. 
# Сколько наблюдений попало в тестовую выборку?

X = df.drop(['satisfaction'], axis=1)
y = df['satisfaction']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state=26)
X_test.shape

(20781, 25)

In [77]:
# 6.6
# Теперь нам необходимо реализовать масштабирование данных. Для этого обучите на обучающей выборке метод StandardScaler() 
# и с помощью него преобразуйте и обучающую, и тестовую выборки. Не забудьте, что целевую переменную обрабатывать не нужно.

# Примечание. Отметим, что если бы дальше мы работали только с деревьями, масштабирование бы не требовалось. 
# Однако мы реализуем его, чтобы можно было обучать и другие модели и сравнивать полученные результаты.

# В качестве ответа введите самое первое значение из матрицы преобразованных признаков тестовой выборки. 
# Округлите значение до двух знаков после точки-разделителя.

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train, y_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_test_scaled[0]

array([ 0.94082514, -1.0182236 ,  0.47381845,  0.37290362,  0.67016452,
        2.69748792, -0.54752968, -0.69553636,  0.17491054, -0.76416846,
        0.60146049,  1.29843847,  1.18407603,  0.48374602,  0.47926399,
        0.494379  ,  0.31485443,  1.34221146,  0.30683708, -0.21471839,
       -0.38594265, -0.36434834,  1.04624209, -0.90523003, -0.27897268])

In [78]:
# 6.7
# Перейдём к обучению моделей. В качестве первой модели возьмём самую простую — логистическую регрессию. 
# Мы делаем это для того, чтобы потом сравнивать с ней полученные результаты: 
# так вы сможете выяснить, насколько ансамбли смогут улучшить точность прогноза.

# Обучите логистическую регрессию с параметрами по умолчанию на наших данных. В качестве ответа введите значение метрики f1_score. 
# Ответ округлите до трёх знаков после точки-разделителя.

model = LogisticRegression()
model.fit(X_train_scaled,y_train)
y_pred_log = model.predict(X_test_scaled)
print("log f1= ", metrics.f1_score(y_test, y_pred_log))

log f1=  0.8546883773161146


In [80]:
# 6.8
# Теперь перейдём к бустингу. Начнём с обучения первой модели — AdaBoost. 
# В качестве базовой модели для неё возьмите решающее дерево с параметром random_state = 26.

# Обучите AdaBoost, зафиксировав random_state со значением 26 и задав темп обучения 0.01. 
# В качестве ответа введите значение метрики f1_score. Ответ округлите до трёх знаков после точки-разделителя.
from sklearn.ensemble import AdaBoostClassifier

model = AdaBoostClassifier(
    base_estimator=DecisionTreeClassifier(random_state = 26),
    learning_rate = 0.01,
    random_state=26
)
model.fit(X_train_scaled,y_train)
y_pred_ada = model.predict(X_test_scaled)
print("ada f1= ", metrics.f1_score(y_test, y_pred_ada))


ada f1=  0.9404794558121674


In [82]:
# 6.9
# Перейдем к следующему алгоритму — градиентному бустингу.

# Будем настраивать количество деревьев и темп обучения, делая перебор по следующей сетке:

# params = {"n_estimators":2**np.arange(8), "learning_rate":0.1**np.arange(3)}
# Используйте для поиска оптимальных параметров GridSearchCV, 
# а для ускорения работы алгоритма задайте параметр кросс-валидации, равный 3.

# Какое наибольшее значение метрики f1_score получилось? Ответ округлите до трёх знаков после точки-разделителя.

# Примечание. Необходимо указать лучший результат в методе GridSearchCV на тренировочных данных.
from sklearn.ensemble import GradientBoostingClassifier

params = {"n_estimators":2**np.arange(8), "learning_rate":0.1**np.arange(3)}
grid = GridSearchCV(
    estimator=GradientBoostingClassifier(),
    param_grid = params,
    cv= 3,
    scoring='f1'
)
grid.fit(X_train_scaled, y_train)
print( grid.best_score_)

0.9491552925864454


In [83]:
# 6.10
# Обучите алгоритм XGBoost. Так как он достаточно мощный «из коробки», определите его с параметрами по умолчанию, 
# только задайте random_state = 26. 
# Какое значение метрики f1_score получилось? Ответ округлите до трёх знаков после точки-разделителя.

# Подсказка. Для того чтобы обучить алгоритм XGBoost для решения задачи классификации, 
# вам понадобится XGBClassifier из библиотеки xgboost, установленной ранее.
# Вся дальнейшая последовательность действий (обучение модели, предсказание, оценка качества) идентична другим алгоритмам, 
# например логистической регрессии.

from xgboost import XGBClassifier

model = XGBClassifier(
    random_state=26
)
model.fit(X_train_scaled,y_train)
y_pred_xgb = model.predict(X_test_scaled)
print("xgb f1= ", metrics.f1_score(y_test, y_pred_xgb))

xgb f1=  0.9579785161685312


In [84]:
# 6.11
# Обучите алгоритм CatBoost. Как и XGBoost, будем обучать его с настройками по умолчанию и заданным random_state = 26. 
# Какое значение метрики f1_score получилось? Ответ округлите до трёх знаков после точки-разделителя.

# Подсказка. Для того чтобы обучить алгоритм CatBoost, вам понадобится CatBoostClassifier() из библиотеки catboost, установленной ранее. 
# Вся дальнейшая последовательность действий (обучение модели, предсказание, оценка качества) идентична другим алгоритмам, 
# например логистической регрессии.

from catboost import CatBoostClassifier

model = CatBoostClassifier(
    random_state=26
)
model.fit(X_train_scaled,y_train)
y_pred_cat = model.predict(X_test_scaled)
print("cat f1= ", metrics.f1_score(y_test, y_pred_cat))

Learning rate set to 0.068023
0:	learn: 0.6018180	total: 66.2ms	remaining: 1m 6s
1:	learn: 0.5020757	total: 74.9ms	remaining: 37.4s
2:	learn: 0.4472475	total: 86.1ms	remaining: 28.6s
3:	learn: 0.4028686	total: 93.7ms	remaining: 23.3s
4:	learn: 0.3674732	total: 102ms	remaining: 20.4s
5:	learn: 0.3397844	total: 110ms	remaining: 18.3s
6:	learn: 0.3121216	total: 118ms	remaining: 16.8s
7:	learn: 0.2917501	total: 127ms	remaining: 15.7s
8:	learn: 0.2749040	total: 135ms	remaining: 14.8s
9:	learn: 0.2575191	total: 143ms	remaining: 14.2s
10:	learn: 0.2473690	total: 151ms	remaining: 13.6s
11:	learn: 0.2377533	total: 160ms	remaining: 13.2s
12:	learn: 0.2279311	total: 168ms	remaining: 12.8s
13:	learn: 0.2212511	total: 177ms	remaining: 12.4s
14:	learn: 0.2100357	total: 185ms	remaining: 12.2s
15:	learn: 0.2025732	total: 194ms	remaining: 11.9s
16:	learn: 0.1942301	total: 205ms	remaining: 11.9s
17:	learn: 0.1877937	total: 215ms	remaining: 11.8s
18:	learn: 0.1832380	total: 224ms	remaining: 11.6s
19:	lea

In [90]:
# 6.12
# Выведите матрицу ошибок для алгоритма, который получил наилучшие показатели качества модели на обучающей выборке 
# (будем считать, что оцениваем по f1_score). 
from catboost import Pool, CatBoostClassifier
from catboost.utils import get_confusion_matrix

cm = get_confusion_matrix(model, Pool(X_train_scaled, y_train))
cm

array([[46651.,   555.],
       [ 1279., 34638.]])

In [92]:
# 6.13
# Оцените важность признаков для модели из предыдущего задания. 
# Отметьте признак, который оказывает наибольшее влияние на значение целевой переменной:

feature_names = [x for x in df if x != 'satisfaction']
pd.DataFrame({'feat': feature_names,
              'coef': model.feature_importances_}).sort_values(by='coef', ascending=False)

,feat,coef
6,Inflight wifi service,25.247444
4,Type of Travel,17.655215
2,Customer Type,7.220277
11,Online boarding,6.980702
22,Class_Business,5.733204
17,Checkin service,4.062630
3,Age,3.776452
16,Baggage handling,3.663133
9,Gate location,3.404037
12,Seat comfort,3.004161


# 7. Стекинг

In [98]:
df = pd.read_csv('data/Frogs_MFCCs.csv')
X = df.drop(['Family', 'Genus', 'Species', 'RecordID'], axis = 1)
y = df['Family'].map({'Dendrobatidae' : 1, 'Leptodactylidae':0, 'Hylidae':0,'Bufonidae':0})
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2,random_state = 31)

In [99]:
# 7.5
# Для начала обучите на наших данных случайный лес с десятью решающими деревьями. 
# Воспользуйтесь параметрами по умолчанию. В качестве значения random_state возьмите число 42.

# Оцените значение F1-меры и введите его в качестве ответа, предварительно округлив до двух знаков после точки-разделителя.
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=10,
    random_state=42
)
rf.fit(X_train,y_train)
y_pred_rf = rf.predict(X_test)
print("rf f1= ", metrics.f1_score(y_test, y_pred_rf))

rf f1=  0.9726775956284154


In [102]:
# 7.6
# Теперь попробуем улучшить качество нашего предсказания за счёт использования стекинга.

# В качестве базовых моделей выберите следующие:

# случайный лес с десятью деревьями, random_state = 31;
# KNN, количество соседей = 11;
# наивный байесовский классификатор с параметрами по умолчанию, в качестве метода возьмите GaussianNB().
# В качестве метамодели выберите логистическую регрессию.

# Обучите модели и сделайте предсказание целевой метки для тестового набора данных.

# Рассчитайте  -меру для тестового набора данных и введите её в качестве ответа, 
# предварительно округлив до двух знаков после точки-разделителя.
from sklearn.ensemble import StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

models = [('rf', rf), 
          ('knn', KNeighborsClassifier(n_neighbors=11)), 
          ('nbc',GaussianNB())]

stack = StackingClassifier(
    models,
    final_estimator= LogisticRegression()
                           )
stack.fit(X_train,y_train)
y_pred_stack = stack.predict(X_test)
print("stack f1= ", metrics.f1_score(y_test, y_pred_stack))


stack f1=  0.989247311827957
